# Neural Network for all lipids

### Import

In [45]:
import os
import random
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_addons as tfa
from keras.callbacks import EarlyStopping
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout, Activation
from keras.initializers import he_normal
from keras.regularizers import l2

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import KFold

### Reproducibility

In [46]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

### Data loading

In [47]:
# Dataset path 
dataset_dir = 'data/processed_data'
# Training loading
training_input_path = os.path.join(dataset_dir, 'train_features.parquet')
training_output_path = os.path.join(dataset_dir, 'train_targets.parquet')
# Testing loading
testing_input_path = os.path.join(dataset_dir, 'test_features.parquet')
testing_output_path = os.path.join(dataset_dir, 'test_targets.parquet')

In [48]:
# Load dataset into Pandas dataframes
training_input = pd.read_parquet(training_input_path)
training_output = pd.read_parquet(training_output_path)

testing_input = pd.read_parquet(testing_input_path)
testing_output = pd.read_parquet(testing_output_path)

lipids_to_drop = ['LPC O-18:3', 'Cer 36:1', 'PE(30:1) ', 'PE 34:2', 'PA(P-38:6)\xa0',
 'PI-Cer(t30:2)\xa0', 'PE 36:3', 'PE O-39:6', 'PA 40:6', 'PA 42:4',
 'PE(O-40:6)\xa0', 'PE(42:6)\xa0', 'PE(40:8)\xa0', 'PGP(34:1) ', 'PE(44:11(OH))\xa0',
 'PS(40:4) ', 'PIP(O-36:5)\xa0', 'PI-Cer(t30:0)\xa0']

training_output = training_output.drop(lipids_to_drop, axis=1)

# Number of output nodes (lipids) for the model
OUTPUT_NODES = training_output.shape[1]
# Number of input nodes (genes) for the model
input_dim = training_input.shape[1]
# Batch size
batch_size = 32

In [49]:
# Create a Sequential model
def build_model(summary=False):
    model = Sequential()

    # Add the input layer with 500 nodes
    model.add(Dense(128, input_dim=500, kernel_initializer=he_normal(seed=seed), activation='gelu'))
    model.add(Dense(64, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    model.add(Dense(32, activation='gelu', kernel_initializer=he_normal(seed=seed)))
    # Add the output layer with OUTPUT_NODES nodes (for multiple regression)
    model.add(Dense(OUTPUT_NODES, activation='sigmoid'))

    if summary:
        # Display the model summary
        model.summary()

    return model

# Print the model summary
build_model(True)

Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_120 (Dense)           (None, 128)               64128     
                                                                 
 dense_121 (Dense)           (None, 64)                8256      
                                                                 
 dense_122 (Dense)           (None, 32)                2080      
                                                                 
 dense_123 (Dense)           (None, 138)               4554      
                                                                 
Total params: 79,018
Trainable params: 79,018
Non-trainable params: 0
_________________________________________________________________


In [50]:
# Calculate loss for each output node (lipid)
def calculate_losses(lipid_true: pd.DataFrame, lipid_pred: np.ndarray):
    losses = np.zeros((OUTPUT_NODES,))
    for i in range(OUTPUT_NODES):
        y_pred = lipid_pred[:, i]
        y_true = lipid_true.iloc[:, i]
        losses[i] = mean_squared_error(y_true, y_pred)
    return losses

# Calculate loss for each output node (lipid)
def calculate_r2s(lipid_true: pd.DataFrame, lipid_pred: np.ndarray):
    r2s = np.zeros((OUTPUT_NODES,))
    for i in range(OUTPUT_NODES):
        y_pred = lipid_pred[:, i]
        y_true = lipid_true.iloc[:, i]
        r2s[i] = r2_score(y_true, y_pred)
    return r2s

In [51]:
# Use early stopping on the validation
early_stopping = EarlyStopping(monitor='val_loss',  # Metric chose is validation loss (MSE)
                               patience=10,         # Number of epochs with no improvement after which training stops
                               restore_best_weights=True)  # Restores model weights from the epoch with the best value of the monitored metric

In [52]:
lipid_names = list(map(lambda s: s.strip(), training_output.columns.values))
lipids_metrics_avg = pd.DataFrame(columns=['Loss', 'R2', 'R2_no_scale'], index=lipid_names)

print(f'Start training')
num_folds = 5
# Define the K-fold Cross Validator
k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)

# Metrics for each fold
loss_per_fold = np.zeros((num_folds,len(lipid_names)))
r2_per_fold = np.zeros((num_folds,len(lipid_names)))
r2_per_fold_no_scale = np.zeros((num_folds,len(lipid_names)))

# K-fold Cross Validation model evaluation
fold_no = 1
models = []
split_indices = []
final_losses = []
final_r_score = []

for input_indices, output_indices in k_fold.split(training_input, training_output):
    split_indices.append(output_indices)
    # Build the model
    model = build_model()

    # Fit and transform the data
    scaler = MinMaxScaler()
    scaler.fit(training_output.iloc[input_indices])
    training_output_scale = pd.DataFrame(scaler.transform(training_output.iloc[input_indices]), columns=training_output.columns, index=training_output.iloc[input_indices].index)
    validation_output_scale = pd.DataFrame(scaler.transform(training_output.iloc[output_indices]), columns=training_output.columns, index=training_output.iloc[output_indices].index)

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error',
                  metrics=[metrics.mean_squared_error, metrics.mean_absolute_error, tfa.metrics.RSquare(), metrics.mean_absolute_percentage_error])

    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

    # Train the model
    history = model.fit(
        training_input.iloc[input_indices], training_output_scale,
        epochs=200,  # Adjust the number of epochs as needed
        batch_size = 32,
        validation_data=(training_input.iloc[output_indices], validation_output_scale),
        callbacks=[early_stopping],
        shuffle=True
    ).history

    final_losses.append(model.evaluate(training_input.iloc[output_indices], validation_output_scale)[0])
    final_r_score.append(model.evaluate(training_input.iloc[output_indices], validation_output_scale)[3])

    # Generate generalization metrics
    prediction = model.predict(training_input.iloc[output_indices])
    losses = calculate_losses(validation_output_scale, prediction)
    r2s = calculate_r2s(validation_output_scale, prediction)
    loss_per_fold[fold_no-1] = losses
    r2_per_fold[fold_no-1] = r2s
    reversed_data = scaler.inverse_transform(prediction)
    r2_per_fold_no_scale[fold_no-1] = calculate_r2s(training_output.iloc[output_indices], reversed_data)

    # Increase fold number
    fold_no = fold_no + 1

mean_loss = loss_per_fold.mean(axis=0)
mean_r2 = r2_per_fold.mean(axis=0)
mean_r2_no_scale = r2_per_fold_no_scale.mean(axis=0)
lipids_metrics_avg['Loss'] = mean_loss
lipids_metrics_avg['R2'] = mean_r2
lipids_metrics_avg['R2_no_scale'] = mean_r2_no_scale
print('#'*72)
print(f'Finish training')

mean = np.mean(np.array(final_losses))
std = np.std(np.array(final_losses))
print(f'The final loss is: {mean} + {std}')
mean = np.mean(np.array(final_r_score))
std = np.std(np.array(final_r_score))
print(f'The final R2 is: {mean} + {std}')

lipids_metrics_avg.to_csv('results/neural_network/lipids_metrics_avg_neural_network.csv')

Start training


/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/pyth

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/200
1565/1565 [==============================] - 9s 5ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - mean_absolute_error: 0.0641 - r_square: -0.1522 - mean_absolute_percentage_error: 9487758.0000 - val_loss: 0.0061 - val_mean_squared_error: 0.0061 - val_mean_absolute_error: 0.0542 - val_r_square: 0.3220 - val_mean_absolute_percentage_error: 7081848.5000
Epoch 2/200
1565/1565 [==============================] - 6s 4ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - mean_absolute_error: 0.0533 - r_square: 0.3510 - mean_absolute_percentage_error: 7055385.5000 - val_loss: 0.0056 - val_mean_squared_error: 0.0056 - val_mean_absolute_error: 0.0518 - val_r_square: 0.3682 - val_mean_absolute_percentage_error: 6631245.5000
Epoch 3/200
1565/1565 [==============================] - 6s 4ms/step - loss: 0.0054 - mean_squared_error: 0.0054 - mean_absolute_error: 0.0509 - r_square: 0.3982 - 

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/200
1565/1565 [==============================] - 11s 6ms/step - loss: 0.0086 - mean_squared_error: 0.0086 - mean_absolute_error: 0.0632 - r_square: -0.0905 - mean_absolute_percentage_error: 9259245.0000 - val_loss: 0.0062 - val_mean_squared_error: 0.0062 - val_mean_absolute_error: 0.0542 - val_r_square: 0.3195 - val_mean_absolute_percentage_error: 6612284.5000
Epoch 2/200
1565/1565 [==============================] - 8s 5ms/step - loss: 0.0058 - mean_squared_error: 0.0058 - mean_absolute_error: 0.0532 - r_square: 0.3566 - mean_absolute_percentage_error: 7028151.0000 - val_loss: 0.0059 - val_mean_squared_error: 0.0059 - val_mean_absolute_error: 0.0539 - val_r_square: 0.3564 - val_mean_absolute_percentage_error: 6886961.0000
Epoch 3/200
1565/1565 [==============================] - 10s 7ms/step - loss: 0.0053 - mean_squared_error: 0.0053 - mean_absolute_error: 0.0509 - r_square: 0.4013 

...

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

------------------------------------------------------------------------
Training for fold 5 ...
Epoch 1/200
1565/1565 [==============================] - 7s 4ms/step - loss: 0.0089 - mean_squared_error: 0.0089 - mean_absolute_error: 0.0635 - r_square: -0.2385 - mean_absolute_percentage_error: 9357295.0000 - val_loss: 0.0060 - val_mean_squared_error: 0.0060 - val_mean_absolute_error: 0.0534 - val_r_square: 0.3159 - val_mean_absolute_percentage_error: 6797538.5000
Epoch 2/200
1565/1565 [==============================] - 6s 4ms/step - loss: 0.0057 - mean_squared_error: 0.0057 - mean_absolute_error: 0.0524 - r_square: 0.3508 - mean_absolute_percentage_error: 6979502.0000 - val_loss: 0.0054 - val_mean_squared_error: 0.0054 - val_mean_absolute_error: 0.0509 - val_r_square: 0.3764 - val_mean_absolute_percentage_error: 6433673.5000
Epoch 3/200
1565/1565 [==============================] - 5s 3ms/step - loss: 0.0052 - mean_squared_error: 0.0052 - mean_absolute_error: 0.0501 - r_square: 0.3980 - 

/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

########################################################################
Finish training
The final loss is: 0.004046402219682932 + 9.764703285896186e-05
The final R2 is: 0.5372694134712219 + 0.004563678135022751


/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/violarenne/opt/anaconda3/envs/ml-project-2-genelipids/lib/python3.10/site-packages/sklearn/utils/validation.py:

In [54]:
# Print k best and worst lipids for average between folds of metrics loss and r2
k = 5

def format_names(df: pd.DataFrame, just=15):
    return ', '.join([name.rjust(just) for name in df.index.values])

def format_values(df: pd.DataFrame, col: str, just=15):
    return ', '.join([f'{val:.5e}'.rjust(just) for val in df[col].values])

best_losses = lipids_metrics_avg.nsmallest(k, 'Loss')
worst_losses = lipids_metrics_avg.nlargest(k, 'Loss')
print("Loss:")
print("  Best:")
print(f"  {format_names(best_losses)}")
print(f"  {format_values(best_losses, 'Loss')}")
print("  Worst:")
print(f"  {format_names(worst_losses)}")
print(f"  {format_values(worst_losses, 'Loss')}")

best_r2s = lipids_metrics_avg.nlargest(k, 'R2')
worst_r2s = lipids_metrics_avg.nsmallest(k, 'R2')
print("R2:")
print("  Best:")
print(f"  {format_names(best_r2s)}")
print(f"  {format_values(best_r2s, 'R2')}")
print("  Worst:")
print(f"  {format_names(worst_r2s)}")
print(f"  {format_values(worst_r2s, 'R2')}")

Loss:
  Best:
          SM 40:1,         SM 42:2,         PE 37:2,         SM 38:1,     HexCer 36:0
      5.28856e-04,     6.12392e-04,     6.72464e-04,     7.16687e-04,     7.73929e-04
  Worst:
          PC 35:0,   PE-Cer(d38:1),        PG(40:3),    PE(40:2(OH)),   PI-Cer(t28:0)
      1.55944e-02,     1.44495e-02,     1.40846e-02,     1.22180e-02,     1.21788e-02
R2:
  Best:
          PC 36:4,        PA(34:1),        PG(34:0),        PG(38:4),         PC 32:0
      8.14353e-01,     8.11972e-01,     8.11657e-01,     8.00914e-01,     7.96386e-01
  Worst:
         PE(36:2),       PE O-38:5,         PC 33:1,       PE O-34:2,         PC 34:2
      8.93798e-02,     1.05395e-01,     1.08360e-01,     1.21242e-01,     1.73695e-01
